In [42]:
import pandas as pd
from statistics import mean

In [43]:
GNN = 0

if GNN == 1:
    model = 'MPNN' #MPNN CGCNN GCN
    ff = 'MMFF94' #UnOp GAFF MMFF94

    folder = rf'C:/Users/nfurth/Desktop/Papers/EACN Paper/Model Results/{model}-{ff}/'
    
    train_file = folder + rf'Test_train_outputs.csv'
    val_file = folder + rf'Test_val_outputs.csv'
    test_file = folder + rf'Test_test_outputs.csv'

    train_outputs_df = pd.read_csv(train_file)
    val_outputs_df = pd.read_csv(val_file)
    test_outputs_df = pd.read_csv(test_file)
    train_val_df = pd.concat([train_outputs_df, val_outputs_df])
if GNN == 0:
    model = 'XGBoost' #XGBoost GP

    folder = rf'C:/Users/nfurth/Desktop/Papers/EACN Paper/Model Results/{model}/'
    train_file = folder + rf'Test_train_outputs-{model}.csv'
    test_file = folder + rf'Test_test_outputs-{model}.csv'

    train_outputs_df = pd.read_csv(train_file)
    test_outputs_df = pd.read_csv(test_file)

eacn_df = pd.read_csv("C:/Users/nfurth/Desktop/Papers/EACN Paper/EACN.csv")

if GNN == 1:
    merged_train_df = pd.merge(train_val_df, eacn_df, left_on='ids', right_on='Molecule')
if GNN == 0:
    merged_train_df = pd.merge(train_outputs_df, eacn_df, left_on='ids', right_on='Molecule')

merged_train_df['Error'] = abs(merged_train_df['target'] - merged_train_df['prediction'])
merged_train_df = merged_train_df.drop(['Molecule', 'Target', 'SMILES'], axis=1)

merged_test_df = pd.merge(test_outputs_df, eacn_df, left_on='ids', right_on='Molecule')
merged_test_df['Error'] = abs(merged_test_df['target'] - merged_test_df['prediction'])
merged_test_df = merged_test_df.drop(['Molecule', 'Target', 'SMILES'], axis=1)

In [44]:
merged_train_df

,ids,target,prediction,Category,Error
0,OctaneNitrile,-1.70,-1.437306,Nitrile,0.262694
1,Pinane,4.10,4.020829,Terpene,0.079171
2,1-Chlorooctane,2.25,1.677104,Halogenated Alkane,0.572896
3,Glycerol Trioctanoate,12.30,12.571815,Ester,0.271815
4,Dipropoxyethane,0.40,0.417994,Ether,0.017994
...,...,...,...,...,...
124,Ethyl Palmitate,6.80,6.888563,Ester,0.088563
125,Menthyl Acetate,-0.10,-0.374155,Terpenoid,0.274155
126,Tripalmitolein,19.00,19.417904,Triglyceride,0.417904
127,TriChloro Ethylene,2.90,2.489105,Halogenated Alkane,0.410895


In [45]:
n = 182
k = 17

In [46]:
category_means = merged_train_df.groupby('Category')['Error'].mean()

merged_train_df = pd.merge(merged_train_df, category_means, on='Category', how='left', suffixes=('', '_mean'))
merged_train_df['Squared_Error_SSW'] = (merged_train_df['Error'] - merged_train_df['Error_mean'])**2
ssw_per_category = merged_train_df.groupby('Category')['Squared_Error_SSW'].sum()

overall_mean = merged_train_df['Error'].mean()

merged_train_df['Squared_Error_SSB'] = (merged_train_df['Error'] - overall_mean)**2
ssb_per_category = merged_train_df.groupby('Category')['Squared_Error_SSB'].sum()

SSW = sum(ssw_per_category)
SSB = sum(ssb_per_category)
print(SSB, '\n', SSW)

print('Training SSB/SSW:', ((SSB/(k-1)))/((SSW/(n-k))))

109.76898221080461 
 97.76585122706459
Training SSB/SSW: 11.578609656042683


In [47]:
category_means = merged_train_df.groupby('Category')['Error'].mean()

merged_test_df = pd.merge(merged_test_df, category_means, on='Category', how='left', suffixes=('', '_mean'))
merged_test_df['Squared_Error_SSW'] = (merged_test_df['Error'] - merged_test_df['Error_mean'])**2
ssw_per_category = merged_test_df.groupby('Category')['Squared_Error_SSW'].sum()

overall_mean = merged_test_df['Error'].mean()

merged_test_df['Squared_Error_SSB'] = (merged_test_df['Error'] - overall_mean)**2
ssb_per_category = merged_test_df.groupby('Category')['Squared_Error_SSB'].sum()

SSW = sum(ssw_per_category)
SSB = sum(ssb_per_category)
print(SSB, '\n', SSW)

print('Testing SSB/SSW:', ((SSB/(k-1)))/((SSW/(n-k))))

144.24905007179942 
 267.03395840145555
Testing SSB/SSW: 5.57070845135374
